In [2]:
import pandas as pd
import spot
import os
import re

In [6]:
timeout =30

In [7]:
telatko = '/home/xschwar3/telatko/telatko.py'

In [8]:
automata_sets = ['delag', 'rabinizer4', 'ltl3tela', 'ltl2tgba']

In [9]:
os.environ['SPOT_TMPDIR'] = '/home/xschwar3/telatko/statistics/march_23/autcross'
os.environ['SPOT_TMPKEEP'] = '1'

In [10]:
def run_autcross(tool):
    !autcross --no-checks --csv=patterns-telatko-{tool}.csv -F patterns-{tool}.hoa 'autfilt %H > %O' '{telatko} -F %H -T {timeout} -L 1 -I -O %O' '{telatko} -F %H -T {timeout} -L 2 -I -O %O' '{telatko} -F %H -T {timeout} -L 3 -I -O %O' '{telatko} -F %H -T {timeout} -I -G -L 3 -O %O' > autcross/{tool}.log 2>&1

In [11]:
for tool in automata_sets:
    run_autcross(tool)

In [14]:
def get_data():
    data = {}

    for tool in automata_sets:
        #print(tool)
        csv = pd.read_csv('patterns-telatko-' + tool + '.csv')

        header = {
            'L0.acc': [],
            'L1.acc': [],
            'L1.time': [],
            'L2.acc': [],
            'L2.time': [],
            'L3.acc': [],
            'L3.time': [],
            'L4.acc': [],
            'L4.time': [],
        }
        i = 0
        for _,r in csv.iterrows():
            #print(i)
            #print(r)
            #header[metrics[i]].append(r['output.acc_sets'])
            header['L' + str(i) + '.acc'].append(r['output.acc_sets'])
            if i > 0:
                header['L' + str(i) + '.time'].append(r['time'])
            i = (i + 1) % 5



        data_dd = pd.DataFrame(data = header)

        data_de = data_dd.dropna()
        #print(data_de)


        data_df = data_de[(data_de['L0.acc'] > 1)]
        data[tool] = data_df
        
        #print(data_df)
    return data



    

In [15]:
def get_table(data):
    sumdata = {
        'tool': [],
        'autcount': [],
        'L0.acc': [],
    }
    for l in range(1, 5):
        for s in ['acc', 'time', 'ratio']:
            sumdata['L' + str(l) + '.' + s] = []

    for tool in automata_sets:
        data_df = data[tool]

        sumdata['tool'].append(tool)
        sd = data_df.sum()
        sumdata['autcount'].append(len(data_df))
        for i in range(0, 5):
            sumdata['L' + str(i) + '.acc'].append(int(sd['L' + str(i) + '.acc']))
            if i > 0:
                t = sd['L' + str(i) + '.time']
                #if i > 1 and i < 4:
                #    t -= sd['L' + str(i - 1) + '.time']
                sumdata['L' + str(i) + '.time'].append(round(t, 1))
                sumdata['L' + str(i) + '.ratio'].append(
                    round(100 - 100 * int(sd['L' + str(i) + '.acc']) / int(sd['L0.acc']), 1)
                )

    #for k in sumdata:
    #    print(k, len(sumdata[k]))
    r = pd.DataFrame(data = sumdata)
    return r

In [17]:
d = get_data()

In [18]:
t= get_table(d)

In [19]:
t

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,129,523,389,737.2,25.6,398,1004.5,23.9,397,7256.8,24.1,385,9035.9,26.4
1,rabinizer4,234,882,552,740.5,37.4,564,924.2,36.1,561,1164.1,36.4,542,2344.8,38.5
2,ltl3tela,91,348,332,454.2,4.6,339,532.9,2.6,326,640.9,6.3,323,1473.7,7.2
3,ltl2tgba,70,198,198,496.7,0.0,198,566.6,0.0,189,838.5,4.5,189,843.3,4.5


Implication symmetry breaking DNF.